In [2]:
import numpy as np
import pandas as pd
import itertools

from scipy import special
from datetime import timedelta
from IPython.display import display

In [4]:
# reading necessary data
fraud_df = pd.read_csv('Fraud_Data.csv',header=0)
ISP_df = pd.read_csv('IpAddress_to_Country.csv',header=0)

In [5]:
# checking out the data types for the fraud dataset
fraud_df.dtypes

id                  int64
cadastro           object
compra             object
valor               int64
id_dispositivo     object
fonte              object
browser            object
genero             object
idade               int64
ip                float64
fraude              int64
dtype: object

In [9]:
# checking for null values for the fraud dataset
for col in fraud_df.columns:
    print('Null values in {}: {}'.format(col, fraud_df[col].isnull().sum()))

Null values in id: 0
Null values in cadastro: 0
Null values in compra: 0
Null values in valor: 0
Null values in id_dispositivo: 0
Null values in fonte: 0
Null values in browser: 0
Null values in genero: 0
Null values in idade: 0
Null values in ip: 0
Null values in fraude: 0


In [10]:
# describing the fraud dataset for the fraud dataset
fraud_df.describe()

,id,valor,idade,ip,fraude
count,151112.000000,151112.000000,151112.000000,1.511120e+05,151112.000000
mean,200171.040970,36.935372,33.140704,2.152145e+09,0.093646
std,115369.285024,18.322762,8.617733,1.248497e+09,0.291336
min,2.000000,9.000000,18.000000,5.209350e+04,0.000000
25%,100642.500000,22.000000,27.000000,1.085934e+09,0.000000
50%,199958.000000,35.000000,33.000000,2.154770e+09,0.000000
75%,300054.000000,49.000000,39.000000,3.243258e+09,0.000000
max,400000.000000,154.000000,76.000000,4.294850e+09,1.000000


In [13]:
# checking out the data types for the IP dataset
ISP_df.dtypes

limite_inferior_ip    float64
limite_superior_ip      int64
pais                   object
dtype: object

In [14]:
# upper and lower IP limits are in different data types.
# checking whether that was merely a dot by the end.

ip_floats = ISP_df.limite_inferior_ip.apply(lambda x: not x.is_integer()).sum()
print('Non-zero decimals in lower IP limits: {}'.format(ip_floats))

Non-zero decimals in lower IP limits: 0


In [18]:
# converting values to integers as there are no relevant decimals
ISP_df.limite_inferior_ip = ISP_df.limite_inferior_ip.astype(int)
ISP_df.dtypes

limite_inferior_ip     int32
limite_superior_ip     int64
pais                  object
dtype: object

In [12]:
# checking for null values for the IP dataset
for col in ISP_df.columns:
    print('Null values in {}: {}'.format(col, ISP_df[col].isnull().sum()))

Null values in limite_inferior_ip: 0
Null values in limite_superior_ip: 0
Null values in pais: 0


In [19]:
# describing the fraud dataset for the IP dataset
ISP_df.describe()

,limite_inferior_ip,limite_superior_ip
count,1.388460e+05,1.388460e+05
mean,-1.214785e+09,2.724557e+09
std,1.563396e+09,8.974979e+08
min,-2.147484e+09,1.677747e+07
25%,-2.147484e+09,1.920008e+09
50%,-2.147484e+09,3.230888e+09
75%,5.228780e+08,3.350466e+09
max,2.113929e+09,3.758096e+09


In [20]:
# strange value at 75% percentile for the lower limit,
# checking whether it is merely a display issue

display(ISP_df[ISP_df.limite_inferior_ip == ISP_df.limite_inferior_ip.max()])
display(ISP_df[ISP_df.limite_superior_ip == ISP_df.limite_superior_ip.max()])

,limite_inferior_ip,limite_superior_ip,pais
37035,2113929216,2130706431,Japan


,limite_inferior_ip,limite_superior_ip,pais
138845,-2147483648,3758096383,Australia


In [26]:
# defining which country an IP belongs to
def ip_to_country(ip):
    tmp = ISP_df[(ISP_df.limite_inferior_ip <= ip)
            & (ISP_df.limite_superior_ip >= ip)].pais
    if tmp.shape[0] == 1:
        return tmp.iloc[0]
    elif tmp.shape[0] < 1:
        return print("Missing")
    elif tmp.shape[0] > 1:
        return print("Error")
    
fraud_df["pais"] = fraud_df.ip.apply(ip_to_country)
fraud_df.head()

Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Missing
Missing
Error
Missing
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Miss

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error


Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Missing
Error
Error
Missing
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error


Error
Error
Error
Missing
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Missing
Missing
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Erro

Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Missing
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Erro

...

Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Missing
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error


Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Missing
Missing
Missing
Error
Error
Error
Error
Error
Missing
Error
Missing
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Error
Error
Error
Error
Error
Error
Error
Error
Missing
Error
Error
Error
Error
Missing
Error
Error
Error
Missing
Error
Missing


,id,cadastro,compra,valor,id_dispositivo,fonte,browser,genero,idade,ip,fraude,pais
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,None
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,None
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,None
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,None


In [27]:
fraud_df.to_csv("fraud_data_country", index=False)
fraud_df = pd.read_csv("fraud_data_country", header=0)